In [1]:
import json
from pathlib import Path
import re

In [2]:
benchmark = ["bt", "cg", "ep", "ft", "is", "lu", "mg", "sp"]
size = "B"
workdir = Path().cwd()

In [3]:
def get_static_info(path):
    info = {}
    with open(path, "r") as f:
        for line in f:
            line = line.strip()
            if line:
                function_match = re.match(r'\[(\d+):([^\]]+)\]', line)
                if function_match:
                    function_id, function_name = function_match.groups()
                    basic_blocks = re.findall(r'\[(\d+):([^\]]*):(\d+)\]', line)
                    for basic_block in basic_blocks:
                        basic_block_id, basic_block_name, basic_block_ir_inst_count = basic_block
                        info[int(basic_block_id)] = {
                            "basic_block_name": basic_block_name,
                            "basic_block_ir_inst_count": int(basic_block_ir_inst_count),
                            "function_name": function_name,
                            "function_id": int(function_id)
                        }
    return info

def form_dict_from_bbv(line):
    line = line.split()
    formed_dict = {}
    for item in line:
        if int(item.split(":")[1]) == 0:
            continue
        formed_dict[int(item.split(":")[0])] = int(item.split(":")[1])
    return formed_dict

def bbv_subtraction(bbv1, bbv2):
    result = {}
    for key in bbv1:
        if key not in bbv2:
            result[key] = bbv1[key]
        else:
            sub= bbv1[key] - bbv2[key]
            if sub > 0:
                result[key] = sub
    return result

def reverse_dict(d):
    return {v: k for k, v in d.items()}

def bbv_dict_to_list(bbv, bbid_map):
    result = [0 for i in range(len(bbid_map))]
    for key, value in bbv.items():
        if key in bbid_map:
            result[bbid_map[key]] = value
        else:
            bbid_map[key] = len(bbid_map)
            result.append(value)
    return result, bbid_map

def get_profiling_info(path):
    info = {}
    with open(path, "r") as f:
        line = f.readline()
        line = f.readline()
        info["roi_begin"] = form_dict_from_bbv(line)
        while line:
            match = re.match(r'Region(\d+)', line.strip())
            if match:
                region_id = int(match.groups()[0])-1
                info[int(region_id)] = {
                    "inst" : int(line.split()[2])
                }
                line = f.readline()
                line = f.readline()
                info[int(region_id)]["bbv"] = form_dict_from_bbv(line)
                line = f.readline()
                line = f.readline()
                info[int(region_id)]["timestamp"] = form_dict_from_bbv(line)
            line = f.readline()
    return info


In [6]:
def form_raw_profiling_info():
    raw_profiling_info = {}
    for bench in benchmark:
        bench_dir = Path(workdir/bench.upper())
        profiler_dir = Path(bench_dir/"profiling")
        matching_files = list(profiler_dir.glob('basic_block_info_output_*.txt'))
        static_info = get_static_info(matching_files[0])
        profiling_info = get_profiling_info(Path(profiler_dir/"aarch64/profiler_output.txt"))
        raw_profiling_info[bench] = {
            "static_info": static_info,
            "profiling_info": profiling_info
        }

    with open("raw_profiling_info.json", "w") as f:
        json.dump(raw_profiling_info, f, indent=4)


In [7]:
form_raw_profiling_info()
raw_profiling_info = json.load(open("raw_profiling_info.json", "r"))


In [8]:

def form_basic_block_matrix_info():
    basic_block_matrix_info = {}
    for bench in benchmark:
        bench_raw_profiling_info = raw_profiling_info[bench]["profiling_info"]
        matrix = []
        total_region = len(bench_raw_profiling_info.keys()) - 1
        region_0_bbv = bbv_subtraction(bench_raw_profiling_info["0"]['bbv'],bench_raw_profiling_info["roi_begin"])
        bbid_map = {}
        region_0_bbv, bbid_map = bbv_dict_to_list(region_0_bbv, bbid_map)
        matrix.append(region_0_bbv)
        for i in range(1, total_region):
            region_i_bbv = bbv_subtraction(bench_raw_profiling_info[str(i)]["bbv"], bench_raw_profiling_info[str(i-1)]["bbv"])
            region_i_bbv, bbid_map = bbv_dict_to_list(region_i_bbv, bbid_map)
            matrix.append(region_i_bbv) 
        for bbv in matrix:
            if len(bbv) != len(bbid_map.keys()):
                bbv.extend([0 for i in range(len(bbid_map.keys()) - len(bbv))])
        reverse_bbid_map = reverse_dict(bbid_map)
        for bbv in matrix:
            for index, value in enumerate(bbv):
                bbid = reverse_bbid_map[index]
                # print(raw_profiling_info[bench]["static_info"][str(bbid)])
                bbv[index] = value * raw_profiling_info[bench]["static_info"][str(bbid)]["basic_block_ir_inst_count"]
        basic_block_matrix_info[bench] = {
            "matrix": matrix,
            "bbid_map": bbid_map
        }
    with open("basic_block_matrix_info.json", "w") as f:
        json.dump(basic_block_matrix_info, f, indent=4)
form_basic_block_matrix_info()
basic_block_matrix_info = json.load(open("basic_block_matrix_info.json", "r"))

In [9]:
def find_marker(timestamp, bbv, range):
    candidate = []
    for bbid, time in timestamp.items():
        if time >= range:
            candidate.append(bbid)
    if len(candidate) == 0:
        print(timestamp)
    winner = candidate[0]
    count = bbv[winner]
    for bbid in candidate:
        if bbv[bbid] < count:
            winner = bbid
            count = bbv[bbid]
    return winner

In [10]:
def form_marker_info():
    marker_info = {}
    for bench in benchmark:
        bench_raw_profiling_info = raw_profiling_info[bench]["profiling_info"]
        bench_static_info = raw_profiling_info[bench]["static_info"]
        bench_marker_info = {}
        for rid in range(2, len(bench_raw_profiling_info.keys())-1):
            warmup_start_rid = rid - 2
            start_marker_rid = rid - 1
            end_marker_rid = rid
            time_range = bench_raw_profiling_info[str(warmup_start_rid)]["inst"] - 100000
            warmup_marker_bbid = find_marker(bench_raw_profiling_info[str(warmup_start_rid)]["timestamp"], bench_raw_profiling_info[str(warmup_start_rid)]["bbv"], time_range)
            start_warmup_bbv_diff = bbv_subtraction(bench_raw_profiling_info[str(start_marker_rid)]["bbv"], bench_raw_profiling_info[str(warmup_start_rid)]["bbv"])
            time_range = bench_raw_profiling_info[str(start_marker_rid)]["inst"] - 100000
            start_marker_bbid = find_marker(bench_raw_profiling_info[str(start_marker_rid)]["timestamp"], start_warmup_bbv_diff, time_range)
            end_start_bbv_diff = bbv_subtraction(bench_raw_profiling_info[str(end_marker_rid)]["bbv"], bench_raw_profiling_info[str(start_marker_rid)]["bbv"])
            time_range = bench_raw_profiling_info[str(end_marker_rid)]["inst"] - 100000
            end_marker_bbid = find_marker(bench_raw_profiling_info[str(end_marker_rid)]["timestamp"], end_start_bbv_diff, time_range)
            warmup_marker_count = bench_raw_profiling_info[str(warmup_start_rid)]["bbv"][warmup_marker_bbid]
            start_marker_count = bench_raw_profiling_info[str(start_marker_rid)]["bbv"][start_marker_bbid]
            end_marker_count = bench_raw_profiling_info[str(end_marker_rid)]["bbv"][end_marker_bbid]
            warmup_marker_function_id = bench_static_info[str(warmup_marker_bbid)]["function_id"]
            start_marker_function_id = bench_static_info[str(start_marker_bbid)]["function_id"]
            end_marker_function_id = bench_static_info[str(end_marker_bbid)]["function_id"]
            bench_marker_info[rid] = {
                "warmup_marker_bbid": warmup_marker_bbid,
                "warmup_marker_count": warmup_marker_count,
                "warmup_marker_function_id": warmup_marker_function_id,
                "start_marker_bbid": start_marker_bbid,
                "start_marker_count": start_marker_count,
                "start_marker_function_id": start_marker_function_id,
                "end_marker_bbid": end_marker_bbid,
                "end_marker_count": end_marker_count,
                "end_marker_function_id": end_marker_function_id
            }
        marker_info[bench] = bench_marker_info
    with open("marker_info.json", "w") as f:
        json.dump(marker_info, f, indent=4)

form_marker_info()
